In [1]:
import os
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import requests
import time

In [2]:
#variables
user = os.getenv('glassdoor_user')
password = os.getenv('glassdoor_pass')

In [3]:
options = Options()
#options.add_argument("start-maximized")
#options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.get("https://www.glassdoor.com")



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Driver [C:\Users\VM661TA\.wdm\drivers\chromedriver\win32\99.0.4844.51\chromedriver.exe] found in cache


In [4]:
#login button xpath: //*[@id="SiteNav"]/nav/div[2]/div/div/div/button
login_button = driver.find_element(By.XPATH,'//*[@id="SiteNav"]/nav/div[2]/div/div/div/button')
login_button.click()

In [5]:
#gmail user field xpath: //*[@id="identifierId"]
gmail_user_field = driver.find_element(By.XPATH,'//*[@id="modalUserEmail"]')
gmail_user_field.send_keys(user)

In [6]:
#password field xpath: //*[@id="modalUserPassword"]
pass_field = driver.find_element(By.XPATH, '//*[@id="modalUserPassword"]')
pass_field.send_keys(password)

In [7]:
#login button xpath
login_button = driver.find_element(By.XPATH, '//*[@id="LoginModal"]/div/div/div[2]/div[2]/div[2]/div/div/form/div[3]/button/span')
login_button.click()

In [8]:
time.sleep(5)
driver.get('https://www.glassdoor.com.ar/Explorar/buscar-empresas.htm')
user_agent = {'User-agent': 'Mozilla/5.0'}

In [9]:
reviews_list = []
pages = driver.find_elements(By.CLASS_NAME, 'page')
#I don't use 'page in pages' here because the instance of pages changes when page is refreshed
for i in range(len(pages)):
    pages = driver.find_elements(By.CLASS_NAME, 'page')
    pages[i].click()
    time.sleep(5)
    url = driver.current_url
    response = requests.get(url, headers = user_agent)
    soup = BeautifulSoup(response.text, 'html.parser')
    #make a list of all the company review links on the current page
    links = soup.find_all('a',{'data-test':'cell-Reviews-url'})
    link_list = ['https://www.glassdoor.com.ar/' + link['href'] for link in links]
    for link in link_list:
        driver.get(link)
        tabs = driver.find_elements(By.CLASS_NAME, 'subtle')
        for tab in tabs:
                if tab.text == 'Evaluaciones':
                    tab.click()
                    #extract all positive and negative reviews from each company
                    review_pages = driver.find_elements(By.CLASS_NAME, 'page')
                    #for every page in the review section
                    for j in range(len(review_pages)):
                        time.sleep(5)
                        review_pages = driver.find_elements(By.CLASS_NAME, 'page')
                        driver.execute_script("arguments[0].click();", review_pages[j])
                        url = driver.current_url
                        response = requests.get(url, headers=user_agent)
                        soup = BeautifulSoup(response.text, 'html.parser')
                        company_name = soup.find('title').text.split('|')[0]
                        #rating xpath: //*[@id="EmpStats"]/div/div[1]/div/div/div
                        rating = driver.find_element(By.XPATH, '//*[@id="EmpStats"]/div/div[1]/div/div/div').text
                        all_reviews = soup.find_all('div', class_='mx-0') # later need to filter and keep only the even index of this class, the rest come blank
                        for review in all_reviews:
                            review_dict = {}
                            #find the pros and cons for each review
                            pro = review.findChild('span',{'data-test':'pros'}).text if review.findChild('span',{'data-test':'pros'}) else ''
                            con = review.findChild('span',{'data-test':'cons'}).text if review.findChild('span',{'data-test':'cons'}) else ''
                            review_dict['company'] = company_name
                            review_dict['pros'] = pro
                            review_dict['cons'] = con
                            review_dict['rating'] = rating
                            reviews_list.append(review_dict)
                    break #this breaks the tab loop
    driver.get('https://www.glassdoor.com.ar/Explorar/buscar-empresas.htm')

In [10]:
len(reviews_list)

3000

In [11]:
# keep only the even indexs
reviews_list = [review for review in reviews_list if reviews_list.index(review) % 2 == 0]

In [12]:
df = pd.DataFrame(reviews_list)
df

,company,pros,cons,rating
0,Evaluaciones de Accenture,Remoto\r\nSueldo bueno\r\nTe dan compu y auric...,No aumentan el sueldo muy seguido y los bonos ...,"4,0"
1,Evaluaciones de Accenture,"buffet, gimansio, internet, maasjes, tarjeta a...","sueldo, competencia, computadora, oficina lejo...","4,0"
2,Evaluaciones de Accenture,Flexibilidad laboral y capacitación continua,Las compensaciones económicas serían mejor si ...,"4,0"
3,Evaluaciones de Accenture,Cultura de trabajo y Beneficios corporativos,Salario no competitivo en el mercado,"4,0"
4,Evaluaciones de Accenture,Es un lugar con muy buen clima laboral.,Los sueldos no son competentes en referencia a...,"4,0"
...,...,...,...,...
1495,Evaluaciones de Cognizant Technology Solutions,balance entre vida personal y trabajo es buena,las posibilidades de promociones no son buenas,"3,8"
1496,Evaluaciones de Cognizant Technology Solutions,"Tenés cobertura social, osde 210 y descuentos ...",En la empresa hay poco crecimiento si entrás p...,"3,8"
1497,Evaluaciones de Cognizant Technology Solutions,"Buenos salarios, acordes al mercado, si entrás...","Sólo para cumplir con tu trabajo, te piden un ...","3,8"
1498,Evaluaciones de Cognizant Technology Solutions,"Benefícios y aprendizado, con cursos y compañe...","No hay desventajas, es un buen empleo","3,8"
